In [2]:
from transformers import pipeline

2022-11-30 22:52:50.073019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 22:52:50.461417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 22:52:50.461442: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-30 22:52:50.528432: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-30 22:52:51.819612: W tensorflow/stream_executor/platform/de

1. Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.
1. Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository. 

In [3]:
models = [
    ('bert-base-multilingual-cased', '[MASK]'),
    ('xlm-roberta-base', '<mask>'),
    ('distilbert-base-multilingual-cased', '[MASK]')
]

unmaskers = list(
    map(lambda model, mask: (model, pipeline('fill-mask', model=model), mask), *zip(*models))
)


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [7]:
unmaskers_x, masks_x = zip(*unmaskers)
names_x, _ = zip(*models)


In [9]:
unmaskers = list(zip(names_x, unmaskers_x, masks_x))
unmaskers

[('bert-base-multilingual-cased',
  '[MASK]'),
 ('xlm-roberta-base',
  '<mask>'),
 ('distilbert-base-multilingual-cased',
  '[MASK]')]

In [26]:
MASK = '[[mask]]'

def test_sentences(sentences: list[str]):
    for sentence in sentences:  
        print('#'*20)
        for name, unmasker, mask in unmaskers:
            sentence_m = sentence.replace(MASK,mask)
            print(f'{name}:', end='')
            for result in unmasker(sentence_m):
                print(f'\t{result["sequence"]}', end='')
            print()

3. Devise a method to test if the langage model understands Polish cases. E.g. testing for *nominal case* could be expressed as "Warszawa to największe `[MASK]`", and the masked word should be in nominative case. Create sentences for each case.

In [32]:
sentences = [
    f'{MASK} jest popularnym owocem', # mianownik
    f'Nie ma tutaj z nami {MASK}', # dopełniacz
    f'Przyglądam się {MASK} od godziny', # celownik
    f'Czy widziałeś ostatnio {MASK}', # biernik
    f'Obszar gnuśności zalany {MASK}', # narzędnik
    f'Rozmawialiśmy z nimi o {MASK}', # miejscownik
    f'{MASK} dodaj mi skrzydła', # wołacz
]

sentences_adj = [
    f'Czerwony {MASK} jest popularnym owocem', # mianownik
    f'Nie ma tutaj z nami naszego {MASK}', # dopełniacz
    f'Przyglądam się pięknemu {MASK} od godziny', # celownik
    f'Czy widziałeś ostatnio niebieski {MASK}', # biernik
    f'Poszedłem na spacer z moim {MASK}', # narzędnik
    f'Rozmawialiśmy z nimi o pięknych {MASK}', # miejscownik
    f'{MASK} dodaj mi skrzydła', # wołacz
]

In [33]:
test_sentences(sentences)

####################
bert-base-multilingual-cased:	jest jest popularnym owocem	nie jest popularnym owocem	- jest popularnym owocem	ta jest popularnym owocem	, jest popularnym owocem
xlm-roberta-base:	Nie jest popularnym owocem	nie jest popularnym owocem	Dlaczego jest popularnym owocem	To jest popularnym owocem	który jest popularnym owocem
distilbert-base-multilingual-cased:	Herb jest popularnym owocem	Nie jest popularnym owocem	Obecnie jest popularnym owocem	Miasto jest popularnym owocem	Film jest popularnym owocem
####################
bert-base-multilingual-cased:	Nie ma tutaj z nami.	Nie ma tutaj z nami,	Nie ma tutaj z nami ;	Nie ma tutaj z nami :	Nie ma tutaj z nami?
xlm-roberta-base:	Nie ma tutaj z nami...	Nie ma tutaj z nami.	Nie ma tutaj z nami	Nie ma tutaj z nami!	Nie ma tutaj z nami?
distilbert-base-multilingual-cased:	Nie ma tutaj z nami?	Nie ma tutaj z nami!	Nie ma tutaj z nami :	Nie ma tutaj z nami.	Nie ma tutaj z nami nie
####################
bert-base-multilingual-cased:	P

In [29]:
test_sentences(sentences_adj)

####################
bert-base-multilingual-cased:	Czerwony herb jest popularnym owocem	Czerwony most jest popularnym owocem	Czerwony list jest popularnym owocem	Czerwony pan jest popularnym owocem	Czerwony Herb jest popularnym owocem
xlm-roberta-base:	Czerwony pieprz jest popularnym owocem	Czerwony róż jest popularnym owocem	Czerwony sok jest popularnym owocem	Czerwony pomidor jest popularnym owocem	Czerwony granat jest popularnym owocem
distilbert-base-multilingual-cased:	Czerwony رنگ jest popularnym owocem	Czerwony Ｃ jest popularnym owocem	Czerwony gaz jest popularnym owocem	Czerwony Jerzy jest popularnym owocem	Czerwony музея jest popularnym owocem
####################
bert-base-multilingual-cased:	Nie ma tutaj z nami naszego.	Nie ma tutaj z nami naszego,	Nie ma tutaj z nami naszego :	Nie ma tutaj z nami naszego -	Nie ma tutaj z nami naszego ;
xlm-roberta-base:	Nie ma tutaj z nami naszego domu	Nie ma tutaj z nami naszego serca	Nie ma tutaj z nami naszego Boga	Nie ma tutaj z nami na

4. Devise a method to test long-range relationships such as gender. E.e. you can use two verbs where withe masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [37]:
sentences = [
    f'Poszedłem do pracy i {MASK} tam cały dzień',
    f'Zobaczyła go z daleka i {MASK} za nim',
    f'{MASK} tam i zobacz co się dzieje'
]

sentences_dot = [
    f'Poszedłem do pracy i {MASK} tam cały dzień.',
    f'Zobaczyła go z daleka i {MASK} za nim.',
    f'{MASK} tam i zobacz co się dzieje.'
]

In [35]:
test_sentences(sentences)

####################
bert-base-multilingual-cased:	Poszedłem do pracy i to tam cały dzień	Poszedłem do pracy i jest tam cały dzień	Poszedłem do pracy i tam tam cały dzień	Poszedłem do pracy i gra tam cały dzień	Poszedłem do pracy i ma tam cały dzień
xlm-roberta-base:	Poszedłem do pracy i był tam cały dzień	Poszedłem do pracy i miałem tam cały dzień	Poszedłem do pracy i jestem tam cały dzień	Poszedłem do pracy i będę tam cały dzień	Poszedłem do pracy i pracował tam cały dzień
distilbert-base-multilingual-cased:	Poszedłem do pracy i pracy tam cały dzień	Poszedłem do pracy i pracował tam cały dzień	Poszedłem do pracy i było tam cały dzień	Poszedłem do pracy i przez tam cały dzień	Poszedłem do pracy i 寓 tam cały dzień
####################
bert-base-multilingual-cased:	Zobaczyła go z daleka i była za nim	Zobaczyła go z daleka i to za nim	Zobaczyła go z daleka i miała za nim	Zobaczyła go z daleka i tylko za nim	Zobaczyła go z daleka i ta za nim
xlm-roberta-base:	Zobaczyła go z daleka i była 

In [38]:
test_sentences(sentences_dot)

####################
bert-base-multilingual-cased:	Poszedłem do pracy i to tam cały dzień.	Poszedłem do pracy i jest tam cały dzień.	Poszedłem do pracy i było tam cały dzień.	Poszedłem do pracy i pracy tam cały dzień.	Poszedłem do pracy i tym tam cały dzień.
xlm-roberta-base:	Poszedłem do pracy i był tam cały dzień.	Poszedłem do pracy i miałem tam cały dzień.	Poszedłem do pracy i pracował tam cały dzień.	Poszedłem do pracy i jestem tam cały dzień.	Poszedłem do pracy i stał tam cały dzień.
distilbert-base-multilingual-cased:	Poszedłem do pracy i pracował tam cały dzień.	Poszedłem do pracy i pracy tam cały dzień.	Poszedłem do pracy i školu tam cały dzień.	Poszedłem do pracy i zmarł tam cały dzień.	Poszedłem do pracy i było tam cały dzień.
####################
bert-base-multilingual-cased:	Zobaczyła go z daleka i była za nim.	Zobaczyła go z daleka i to za nim.	Zobaczyła go z daleka i została za nim.	Zobaczyła go z daleka i miała za nim.	Zobaczyła go z daleka i tam za nim.
xlm-roberta-base

5. Check if the model captures real-world knolwedge. For instance a sentence "`[MASK]` wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [42]:
sentences = [
    f'Ziemia to {MASK} planeta od Słońca',
    f'Najwyższą górą na Ziemi jest {MASK}',
    f'Kod genetyczny komórki przechowywany jest w jej {MASK}'
]

sentences_long = [
    f'Ziemia, nazywana niebieską planetą, to {MASK} planeta od Słońca',
    f'Najwyższą górą na Ziemi jest {MASK} o wysokości 8848 m n.p.m.',
    f'Kod genetyczny komórki przechowywany jest w jej {MASK} w postaci łańcuchów DNA'
]

In [43]:
test_sentences(sentences)

####################
bert-base-multilingual-cased:	Ziemia to druga planeta od Słońca	Ziemia to to planeta od Słońca	Ziemia to ta planeta od Słońca	Ziemia to jedna planeta od Słońca	Ziemia to od planeta od Słońca
xlm-roberta-base:	Ziemia to druga planeta od Słońca	Ziemia to większa planeta od Słońca	Ziemia to jedna planeta od Słońca	Ziemia to mniej planeta od Słońca	Ziemia to bliżej planeta od Słońca
distilbert-base-multilingual-cased:	Ziemia to druga planeta od Słońca	Ziemia to pierwsza planeta od Słońca	Ziemia to jedna planeta od Słońca	Ziemia to tylko planeta od Słońca	Ziemia to nazwa planeta od Słońca
####################
bert-base-multilingual-cased:	Najwyższą górą na Ziemi jest :	Najwyższą górą na Ziemi jest.	Najwyższą górą na Ziemi jest to	Najwyższą górą na Ziemi jest góry	Najwyższą górą na Ziemi jest -
xlm-roberta-base:	Najwyższą górą na Ziemi jest:	Najwyższą górą na Ziemi jest...	Najwyższą górą na Ziemi jest...	Najwyższą górą na Ziemi jest	Najwyższą górą na Ziemi jest.
distilbe

In [41]:
test_sentences(sentences_long)

####################
bert-base-multilingual-cased:	Ziemia, nazywana niebieską planetą, to jest planeta od Słońca	Ziemia, nazywana niebieską planetą, to to planeta od Słońca	Ziemia, nazywana niebieską planetą, to druga planeta od Słońca	Ziemia, nazywana niebieską planetą, to więc planeta od Słońca	Ziemia, nazywana niebieską planetą, to : planeta od Słońca
xlm-roberta-base:	Ziemia, nazywana niebieską planetą, to druga planeta od Słońca	Ziemia, nazywana niebieską planetą, to większa planeta od Słońca	Ziemia, nazywana niebieską planetą, to pierwsza planeta od Słońca	Ziemia, nazywana niebieską planetą, to główna planeta od Słońca	Ziemia, nazywana niebieską planetą, to jedna planeta od Słońca
distilbert-base-multilingual-cased:	Ziemia, nazywana niebieską planetą, to druga planeta od Słońca	Ziemia, nazywana niebieską planetą, to pierwsza planeta od Słońca	Ziemia, nazywana niebieską planetą, to jedna planeta od Słońca	Ziemia, nazywana niebieską planetą, to tylko planeta od Słońca	Ziemia, nazyw

6. Take into accout the fact, that causal language models such as PapuGaPT2 or plT5, will only generate continuations of the sentenes, so the
   examples have to be created according to that paradigm.

7. Answer the following questions:
   1. Which of the models produced the best results?
   1. Was any of the models able to capture Polish grammar?
   1. Was any of the models able to capture long-distant relationships between the words?
   1. Was any of the models able to capture world knowledge?
   1. What are the most striking errors made by the models?